In [ ]:
# El siguiente código carga los archivos CSV, elimina duplicados y valores NULL, y muestra las primeras filas de cada DataFrame.
import pandas as pd
import pyodbc
# Conexión a la base de datos BDDEACEROQA
try:
    conn = pyodbc.connect(
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r'SERVER=Fernando-2020\SQLEXPRESS;'
        r'DATABASE=BDDEACEROQA;'
        r'Trusted_Connection=yes;'
    )
    cursor = conn.cursor()
    print("✅ Conexión exitosa a BDDEACEROQA")
except Exception as e:
    print("❌ Error al conectar:", e)

# Cargar los archivos CSV en DataFrames
df_MemTraMDADet = pd.read_csv('MDA_exa.csv')
df_MemTraMTRDet = pd.read_csv('MTR_exa.csv')
df_MemTraTcDet = pd.read_csv('Tc_exa.csv')
df_MemTraTBFinVw = pd.read_csv('tbfin_exa.csv')
# Crear un diccionario para almacenar los DataFrames de los archivos CSV
csv_dataframes = {
    'MDA_exa': df_MemTraMDADet,
    'MTR_exa': df_MemTraMTRDet,
    'Tc_exa': df_MemTraTcDet,
    'tbfin_exa': df_MemTraTBFinVw
}
# Mostrar el número de registros en cada DataFrame
for name, df in csv_dataframes.items():
    print(f"{name}: {len(df)} registros")
# Eliminar registros duplicados en cada DataFrame
for name, df in csv_dataframes.items():
    csv_dataframes[name] = df.drop_duplicates()
# Mostrar el número de registros después de eliminar duplicados
for name, df in csv_dataframes.items():
    print(f"{name}: {len(df)} registros después de eliminar duplicados")

# Eliminar registros con valores NULL en cada DataFrame
for name, df in csv_dataframes.items():
    csv_dataframes[name] = df.dropna()

# Mostrar el número de registros después de eliminar NULLs
for name, df in csv_dataframes.items():
    print(f"{name}: {len(df)} registros después de eliminar NULLs")

# Imporimir las primeras filas de cada DataFrame
for name, df in csv_dataframes.items():
    print(f"\nPrimeras filas de {name}:")
    print(df.head())
    

In [ ]:
# Insertar los DataFrames en las tablas correspondientes
import pandas as pd
import pyodbc

# Conexión a SQL Server
try:
    conn = pyodbc.connect(
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r'SERVER=Fernando-2020\SQLEXPRESS;'
        r'DATABASE=BDDEACEROQA;'
        r'Trusted_Connection=yes;'
    )
    cursor = conn.cursor()
    print("✅ Conexión exitosa a BDDEACEROQA")
except Exception as e:
    print("❌ Error al conectar:", e)
    exit()

# Cargar archivos CSV sin columnas ID
df_MemTraMDADet = pd.read_csv('MDA_exa.csv')  # sin 'idMDA'
df_MemTraMTRDet = pd.read_csv('MTR_exa.csv')  # sin 'idMTR'
df_MemTraTcDet = pd.read_csv('Tc_exa.csv')    # sin 'idTc'
df_MemTraTBFinVw = pd.read_csv('tbfin_exa.csv')

# Diccionario de tablas a insertar (omitimos campos IDENTITY)
csv_dataframes = {
    "MemSch.MemTraMDADet": df_MemTraMDADet,
    "MemSch.MemTraMTRDet": df_MemTraMTRDet,
    "MemSch.MemTraTcDet": df_MemTraTcDet,
    "MemSch.MemTraTBFinVw": df_MemTraTBFinVw  # esta tabla no tiene IDENTITY
}

# Asegurar formato ISO: YYYY-MM-DD
df_MemTraTBFinVw['fecha'] = pd.to_datetime(df_MemTraTBFinVw['fecha'], dayfirst=True).dt.date
# Asegurar que las columnas de fecha estén en formato correcto
for df in csv_dataframes.values():
    if 'fecha' in df.columns:
        df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True).dt.date

# Verificación de nulos
print("\n📊 Verificación de valores nulos:")
for name, df in csv_dataframes.items():
    print(f"\n{name}:")
    print(df.isnull().sum())
    print("Valores nulos encontrados:", df.isnull().values.any())

# Función para insertar sin campos IDENTITY
def insert_dataframe_excluding_identity(df, table_name, cursor, conn, show_sql=False):
    cols = ','.join(df.columns)
    placeholders = ','.join(['?' for _ in df.columns])
    
    print(f"\n📝 Insertando en {table_name} columnas: {cols}")
    print(f"Ejemplo de fila: {df.iloc[0].values.tolist()}")

    insert_query = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"

    for row in df.itertuples(index=False, name=None):
        cursor.execute(insert_query, row)
        
        if show_sql:
            formatted_values = []
            for x in row:
                if pd.isnull(x):
                    formatted_values.append("NULL")
                elif isinstance(x, str):
                    x_clean = x.replace("'", "''")
                    formatted_values.append(f"'{x_clean}'")
                elif isinstance(x, pd.Timestamp):
                    formatted_values.append(f"'{x}'")
                else:
                    formatted_values.append(str(x))
            print(f"INSERT INTO {table_name} ({cols}) VALUES ({', '.join(formatted_values)});")

    conn.commit()

# Insertar datos
try:
    for table_name, df in csv_dataframes.items():
        insert_dataframe_excluding_identity(df, table_name, cursor, conn, show_sql=True)

    print("\n✅ Datos insertados correctamente.")

    # Validación de registros
    print("\n📦 Conteo de registros por tabla:")
    for table_name in csv_dataframes.keys():
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"{table_name}: {count} filas")

except Exception as e:
    print("❌ Error al insertar o consultar:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()
    print("🔒 Conexión cerrada correctamente.")
